# 시청률 데이터 단위별 집계

 LSTM 오토인코더 모형에 feature로 활용하기 위해 시청률을 시계열로 집계한다.

In [1]:
# 모듈 불러오기
import warnings
warnings.filterwarnings(action='ignore')

import pandas as pd
import numpy as np

In [2]:
# 데이터 경로
RATINGS_DATA= './data/2020 빅콘테스트 데이터분석분야-챔피언리그_시청률 데이터.xlsx'

In [3]:
# 데이터 로드
ratings_df = pd.read_excel(RATINGS_DATA, header=1).iloc[:-1, :-1].sort_values(by='시간대').set_index('시간대')
ratings_df

,2019-01-01,2019-01-02,2019-01-03,2019-01-04,2019-01-05,2019-01-06,2019-01-07,2019-01-08,2019-01-09,2019-01-10,...,2019-12-22,2019-12-23,2019-12-24,2019-12-25,2019-12-26,2019-12-27,2019-12-28,2019-12-29,2019-12-30,2019-12-31
시간대,,,,,,,,,,,,,,,,,,,,,
00:00,0.0,0.040,0.0,0.0,0.0,0.0,0.0,0.015,0.0,0.0,...,0.000,0.013,0.0,0.0,0.000,0.000,0.0,0.0,0.0,0.024
00:01,0.0,0.000,0.0,0.0,0.0,0.0,0.0,0.000,0.0,0.0,...,0.000,0.013,0.0,0.0,0.000,0.000,0.0,0.0,0.0,0.024
00:02,0.0,0.023,0.0,0.0,0.0,0.0,0.0,0.000,0.0,0.0,...,0.000,0.013,0.0,0.0,0.000,0.000,0.0,0.0,0.0,0.024
00:03,0.0,0.040,0.0,0.0,0.0,0.0,0.0,0.000,0.0,0.0,...,0.000,0.013,0.0,0.0,0.000,0.000,0.0,0.0,0.0,0.024
00:04,0.0,0.000,0.0,0.0,0.0,0.0,0.0,0.000,0.0,0.0,...,0.018,0.013,0.0,0.0,0.000,0.000,0.0,0.0,0.0,0.024
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
23:55,0.0,0.000,0.0,0.0,0.0,0.0,0.0,0.015,0.0,0.0,...,0.000,0.013,0.0,0.0,0.036,0.000,0.0,0.0,0.0,0.024
23:56,0.0,0.000,0.0,0.0,0.0,0.0,0.0,0.015,0.0,0.0,...,0.000,0.013,0.0,0.0,0.036,0.000,0.0,0.0,0.0,0.024
23:57,0.0,0.040,0.0,0.0,0.0,0.0,0.0,0.015,0.0,0.0,...,0.000,0.013,0.0,0.0,0.050,0.000,0.0,0.0,0.0,0.024


## 하루 단위 시청률

In [19]:
ratings_daily = pd.concat([ratings_df.sum(axis=0), ratings_df.mean(axis=0)], axis=1)\
                .rename(columns={0:'daily_sum', 1:'daily_mean'})
display(ratings_daily)
ratings_daily.to_csv('./data/일일시청률집계(합계,평균).csv', encoding='utf-8-sig', index=False)

,daily_sum,daily_mean
2019-01-01,5.300,0.003681
2019-01-02,8.012,0.005564
2019-01-03,2.492,0.001731
2019-01-04,3.827,0.002658
2019-01-05,2.994,0.002079
...,...,...
2019-12-27,5.585,0.003878
2019-12-28,8.810,0.006118
2019-12-29,6.447,0.004477
2019-12-30,6.589,0.004576


## 시간 단위 시청률

In [61]:
# 일자별 시청률 시간 합계 및 평균
ratings_hourly_sum = ratings_df.groupby(ratings_df.index.str[:2].astype(int), as_index=False).sum().T
ratings_hourly_mean = ratings_df.groupby(ratings_df.index.str[:2].astype(int), as_index=False).mean().T
ratings_hourly_sum.to_csv('./data/시청률시간단위합계.csv', index=False, encoding='utf-8-sig')
ratings_hourly_mean.to_csv('./data/시청률시간단위평균.csv', index=False, encoding='utf-8-sig')

,0,1,2,3,4,5,6,7,8,9,...,14,15,16,17,18,19,20,21,22,23
2019-01-01,0.120,0.040,0.014,0.223,0.663,0.451,0.000,0.000,0.035,0.014,...,0.278,0.666,0.223,0.031,0.673,0.084,0.027,0.146,0.202,0.031
2019-01-02,0.273,0.039,0.000,0.015,0.033,0.000,0.037,0.000,0.587,0.120,...,0.404,0.729,0.123,0.126,1.000,0.150,0.045,0.075,0.312,1.015
2019-01-03,0.015,0.004,0.027,0.066,0.086,0.345,0.057,0.145,0.076,0.064,...,0.112,0.000,0.240,0.136,0.133,0.068,0.071,0.008,0.056,0.174
2019-01-04,0.000,0.470,0.069,0.056,0.000,0.000,0.004,0.000,0.212,0.364,...,0.047,0.295,0.172,0.185,0.033,0.010,0.000,0.248,0.704,0.049
2019-01-05,0.170,0.310,0.000,0.000,0.000,0.078,0.026,0.069,0.183,0.090,...,0.026,0.074,0.044,0.097,0.119,0.140,0.000,0.000,0.000,0.140
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2019-12-27,0.268,0.437,0.012,0.016,0.000,0.016,0.031,0.071,0.075,0.672,...,0.742,0.000,0.397,0.076,0.224,0.325,0.521,0.066,0.145,0.036
2019-12-28,0.000,0.000,0.000,0.000,0.000,0.301,0.071,0.248,0.081,0.279,...,1.169,0.934,0.517,0.265,0.147,0.154,0.241,0.830,1.454,0.328
2019-12-29,0.190,0.042,0.054,0.006,0.094,0.086,0.060,0.076,0.419,0.252,...,0.172,0.538,0.150,0.339,0.172,0.629,0.365,0.091,0.461,0.258
2019-12-30,0.020,0.111,0.000,0.114,0.086,0.024,0.025,0.000,0.166,0.855,...,0.324,1.625,0.409,0.075,0.234,0.212,0.095,0.704,0.125,0.336
